In [1]:
import skimage
import numpy as np
import cv2

from sklearn import svm
from sklearn.externals import joblib
from functions import show, put_labels

from constants import *
from skimage.feature import hog

In [2]:
def drop_features(lt,j=2):
    lt = [lt[i] for i in range(len(lt)) if i%j==0]
    return lt

def filter_features(fd):
    return [f for f in fd if len(f)==2048]

def filter_region(region):
    width = region[2] - region[0]
    height = region[3] - region[1]
    # Incorrect region
    if width==0 or height==0:
        print "Incorrect region",region
        return True
    # distorted rects
#     if width / height > 4 or height / width > 4:
#         return True
    return False

In [3]:
class_positives = np.load('cat_vs_dogs_positives.npy')

#get dicts from numpy array
class_positives = class_positives[()]

In [4]:
dog = class_positives['dog']
cat = class_positives['cat']
print 'before filtering',len(dog),len(cat)
dog = filter_features(dog)
cat = filter_features(cat)
print 'after filtering',len(dog),len(cat)

before filtering 538 389
after filtering 538 389


In [5]:
#extract train data
X = []
X.extend(dog)
y = [1]*len(dog)
X.extend(cat)
y.extend([0]*len(cat))   
X = np.array(X)
y = np.array(y)

In [6]:
def extract(image,box=None):
    """Returns hog feature descriptor for an image.
    """
    #print 'initial',image.shape
    #print len(image)
    if box is not None:
        image = image[box[0]:box[2],box[1]:box[3]]
    
    #resize image
    #print image.shape
    #print len(image)
    image = resize(image)
    #print image.shape
    #image = color.rgb2gray(img)
    fd = hog(image, orientations=8, pixels_per_cell=(image.shape[1]/16, image.shape[0]/16),
                    cells_per_block=(1, 1), visualise=False, transform_sqrt=True)
    #print len(fd)
    return fd

def resize(image):
    if image.shape[1]/image.shape[0] >= 2:
        image = cv2.resize(image,(128,64))
    elif float(image.shape[1])/image.shape[0] <= 0.5:
        image = cv2.resize(image,(64,128))
    else:
        image = cv2.resize(image,(128,128))
    return image

In [7]:
#load test data
test_ground_truth = np.load('test_ground_truth.npy')
test_ground_truth[0][1]

[{'dog': (48, 240, 195, 371)}, {'person': (8, 12, 352, 498)}]

In [8]:
#extract test data
d = {'cat':0,'dog':1}
Xtest,ytest = list(),list()
imgs,i_boxes = list(),list()
for i in range(len(test_ground_truth)):
    boxes = list()
    for dt in test_ground_truth[i][1]:
        if 'dog' in dt or 'cat' in dt:
            if filter_region(dt.values()[0]):
                print dt
            else:
                boxes.append(dt)
    if len(boxes)!=0:
        name = test_ground_truth[i][0]
        imgs.append(name)
        i_boxes.append(boxes)
        img = cv2.imread(folder_test + '/' + name,0)
        #print name,boxes
        for box in boxes:
            #print box.values()[0]
            p_box = box.values()[0]
            p_box = p_box[1::-1] + p_box[3:1:-1] 
            Xtest.append(extract(img,p_box))
            ytest.append(d[box.keys()[0]])

In [9]:
len(ytest)

387

In [10]:
print 'Training for cat vs dogs'
print 'Feature Vectors:',len(X)
from sklearn.grid_search import GridSearchCV
print("Fitting the classifier to the training set")
param_grid = {'C': [0.01, 0.1, 1, 10, 100],'gamma':[0.01, 0.1, 1, 10, 100], 'kernel': ['rbf', 'linear']}
clf = GridSearchCV(svm.SVC(probability=True,class_weight='balanced'), param_grid,n_jobs=3)
clf = clf.fit(X, y)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

# clf = svm.SVC(gamma=0.01, C=100,probability=True)
# clf.fit(X, y)

Training for cat vs dogs
Feature Vectors: 927
Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [11]:
# i_boxes = [[(94,1,442,374)],[(126,51,330,308)],[(48,240,195,371),(8,12,352,498)],[(31,30,358,279)],[(63,18,374,500)]]
# imgs = [cv2.imread('Cat vs Dog Data/000442.jpg',0),cv2.imread('Cat vs Dog Data/000011.jpg',0),
#         cv2.imread('000001.jpg',0),cv2.imread('Cat vs Dog Data/000018.jpg',0),
#         cv2.imread('Cat vs Dog Data/000028.jpg',0)]
pred = clf.predict(Xtest)
from sklearn import metrics
print("Confusion matrix:\n%s" %metrics.confusion_matrix(ytest,pred))
print("Accuracy: %0.4f" % metrics.accuracy_score(ytest,pred))
print "{0}".format(metrics.classification_report(ytest,pred))

Confusion matrix:
[[ 89  61]
 [ 66 171]]
Accuracy: 0.6718
             precision    recall  f1-score   support

          0       0.57      0.59      0.58       150
          1       0.74      0.72      0.73       237

avg / total       0.67      0.67      0.67       387



In [20]:
from sklearn.externals import joblib 
joblib.dump(clf, 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl')

['Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_01.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_02.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_03.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_04.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_05.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_06.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_07.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_08.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_09.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_10.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_11.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_12.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_13.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_14.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_15.npy',
 'Cats Vs Dogs Classifier/cat_pos_vs_dog_pos.pkl_16.npy',
 'Cats Vs Dogs Classi

In [22]:
def find_class(clf,imgs,i_boxes):
    # if boxes is None:
    from extract import extract
    d = {0:'cat',1:'dog'}
    for img,boxes in zip(imgs,i_boxes):
        labels = []
        show(img)
        for box in boxes:
            print box
            input = extract(img,box)
#             print clf.predict_proba([input])
            labels.append(d[clf.predict([input])[0]])
        print labels
        put_labels(img,boxes,labels)

500

In [17]:
test_regions = np.load('test_regns.npy')


In [49]:
def filter_detections(detections):
    detections = sorted(detections,key = lambda detection:detection[1])
    
    

In [ ]:
from os import path
for dt in test_regions:
    name = path.join(folder_test,dt.keys()[0])
    img = cv2.imread(name,0)
    regions = dt.values()[0]
    probs,labels = list(),list()
    regions = [region for region in regions if not filter_region(region)]
    for regn in regions:
        fd = extract(img,regn)
        probs.append(max(clf.predict_proba([fd])[0]))
        labels.append(clf.predict([fd]))
    detections = zip(regions,probs,labels)
    detections = filter_detections(detections)

Incorrect region (285, 499, 405, 499)
Incorrect region (285, 499, 405, 499)
Incorrect region (404, 0, 404, 111)
Incorrect region (405, 0, 405, 190)
Incorrect region (405, 0, 405, 189)
Incorrect region (404, 0, 404, 111)
Incorrect region (68, 19, 187, 19)
Incorrect region (67, 6, 207, 6)
Incorrect region (66, 20, 192, 20)
Incorrect region (75, 50, 177, 50)
Incorrect region (71, 152, 209, 152)
Incorrect region (70, 149, 199, 149)
Incorrect region (67, 6, 181, 6)
Incorrect region (68, 19, 208, 19)
Incorrect region (66, 20, 235, 20)
Incorrect region (67, 6, 181, 6)
Incorrect region (89, 23, 273, 23)
Incorrect region (75, 126, 177, 126)
Incorrect region (75, 126, 177, 126)
Incorrect region (70, 149, 198, 149)
Incorrect region (75, 50, 177, 50)
Incorrect region (61, 22, 217, 22)
Incorrect region (66, 20, 219, 20)
Incorrect region (93, 22, 217, 22)
Incorrect region (76, 125, 176, 125)
Incorrect region (71, 152, 209, 152)
Incorrect region (76, 126, 176, 126)
Incorrect region (71, 152, 208, 152

In [ ]:
from skimage.feature import hog
extract(imgs[0],[10,10,100,300])

In [48]:
filter_detections([[1,2],[2,1],[3,5]])

[[2, 1], [1, 2], [3, 5]]
